In [23]:
import sys
import os, shutil
from os.path import join
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np

In [24]:
sys.path.append('../../../..')
from LCBDtools.scripts import argParser

In [25]:
# parameters

nirs_dir = "/data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/"
# psypy_dir = "/data/perlman/moochie/study_data/JEN/task_data/DB_DOS/"
ex_subs = []
participant_num_len = 4
sample_rate = 7.81250
n_blocks = 2

In [26]:
nirs_session_dirs = [os.path.split(d)[0] for d in glob(
    nirs_dir+"**/*_probeInfo.mat",
    recursive=True) \
    if d.strip(nirs_dir).strip("/")[:participant_num_len] not in ex_subs]

In [27]:
nirs_session_dirs = [ses for ses in nirs_session_dirs if "V1" in ses]

In [28]:
print(nirs_session_dirs[0])

/data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7024/V1/2019-12-11_003/Subject1


In [31]:
start_keys = {
#     '7021 V1': (),
    '7031 V1': (52, 684),
    '7033 V1': (17, 656),
#     '7017 V1': (),
    '7022 V1': (25, 650),
#     '7023 V1': (42, 38),
    '7024 V1': (25, 637),
    '7025 V1': (23, 722),
    '7026 V1': (33, 684),
    '7027 V1': (115, 743),
    '7029 V1': (16, 713),
#     '7030 V1': (),
    '7032 V1': (28, 710),
    '7034 V1': (28, 683),
    '7035 V1': (20, 675),
    '7036 V1': (18, 684),
    '7038 V1': (10, 630),
#     '7039 V1': (),
    '7040 V1': (46, 640),
    '7041 V1': (13, 613),
    '7042 V1': (10, 625),
#     '7043 V1': (21, 24)
}

task_lengths = {
    0: (105, 30), # duration, rest following
    1: (120, 15)
}

In [33]:
for ses in tqdm(nirs_session_dirs):
    # should always work, even with multiple-runs. 
    # TODO remember that these exist, where 2 runs are collected
    # for the same dyad of parent/child, for unknown reason. 
    # potentially check for truncated run while parsing this data in other
    # mne processing software
    
#     if ("50170_V0_fNIRS" in ses) or ("50171_V0_fNIRS" in ses):
#         pass
#     else:
#         continue

    subject = ses.strip(nirs_dir).split('/')[0]
    visit = ses.strip(nirs_dir).split('/')[1]

    # check that it's at least in the start keys
    if subject in [key.split(' ')[0] for key in start_keys.keys()]:
        pass
    else:
        print("Subject not found in keys, skipping:", ses)
        continue
        
    starts = start_keys[' '.join(
        ses.strip(nirs_dir).split('/')[:2])]
    
#     print(subject, starts)
    
    og_evt = glob(ses+"/*.evt")
    if len(og_evt) != 1:
        print("Error: found", len(og_evt), "evt files for the given sub / visit.")
        print("Skipping:", ses)
        continue
    else:
        og_evt = og_evt[0]
    
    stims = []
    
    skip = False
    # block 1 and 2 (0 and 1)
    for block in range(len(task_lengths.keys())):
        
        # actual bit stim value
        val = str(block+1) + "\t0"*7
        
        try:
            start = start_keys[' '.join(subject, visit)][block]
            
            
            stims.append((start, val))
            
            # for each subsequent box in run
            for i in range(3):
                # add stim of start time plus num its and spacing (duration + rest)
                stims.append(((start + (sum(task_lengths[block])*i+1))), val)
            
        except:
            print("Subject / visit not in start keys, skipping:", ses)
            skip = True
            continue
                         
    if skip is True:
        continue
                
    shutil.move(
        og_evt,
        og_evt.replace(".evt", "_old.evt"))
                         
    f = open(join(ses, og_evt), 'w')

    for (time, val) in stims:
        f.write(time)
        f.write('\t')
        f.write(val)
        f.write('\n')
                         
    f.close()

100%|██████████| 40/40 [00:00<00:00, 1463.33it/s]

Subject / visit not in start keys, skipping: /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7024/V1/2019-12-11_003/Subject1
Subject / visit not in start keys, skipping: /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7024/V1/2019-12-11_003/Subject1
Subject / visit not in start keys, skipping: /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7024/V1/2019-12-11_003/Subject2
Subject / visit not in start keys, skipping: /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7024/V1/2019-12-11_003/Subject2
Subject / visit not in start keys, skipping: /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7040/V1/2020-03-04_001/Subject1
Subject / visit not in start keys, skipping: /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7040/V1/2020-03-04_001/Subject1
Subject / visit not in start keys, skipping: /data/perlman/moochie/analysis/Neurophotonics/JEN/NIRS_data_clean/7040/V1/2020-03-04_001/Subject2

In [11]:
def timeconvert_psychopy_to_nirstar(
    sample_rate,
    NSstim1_t,
    PSstim1_t,
    PSevent_t):
    
    NSevent_t = NSstim1_t + (PSevent_t - PSstim1_t) * sample_rate
    return NSevent_t